In [1]:
import pandas as pd
from zipfile import ZipFile
import os
import re
import json
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud
from sklearn.preprocessing import OneHotEncoder
import matplotlib.animation as animation
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
sns.set()

[nltk_data] Downloading package wordnet to /Users/aditya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/aditya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aditya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
import os
directory = 'All Data/Raw Journey Data/Until16May2020(no HQ media)/'
rawdata = {}
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        with open(f'{directory}{filename}', 'r') as f:
            readjson = json.load(f)
            rawdata[readjson['id']]=readjson

In [3]:
len(rawdata)

2589

In [4]:
datadf = pd.DataFrame(rawdata).transpose()

In [5]:
datadf = datadf.drop(columns={'label','music_artist','music_title','preview_text','sentiment','type','folder'})
datadf.set_index('id', inplace=True)

In [6]:
for i in range(len(datadf['date_journal'])):
    datadf['date_journal'][i] = dt.datetime.fromtimestamp((datadf['date_journal'][i])/1000).strftime("%Y-%m-%d %H:%M:%S")
    datadf['date_modified'][i] = dt.datetime.fromtimestamp((datadf['date_modified'][i])/1000).strftime("%Y-%m-%d %H:%M:%S")

In [7]:
datadf['no_punc'] = datadf['text']
for i in range(len(datadf['text'])):
    regex= '<.*?>'
    datadf['no_punc'][i] = re.sub(regex, '', (datadf['text'][i]))
    punct_re='[^\w\s]'
    datadf['no_punc'][i] = re.sub(punct_re, ' ', (datadf['no_punc'][i]))
    newline_re='\\n'
    datadf['no_punc'][i] = re.sub(newline_re, ' ', (datadf['no_punc'][i]))
    newline_re2='nbsp'
    datadf['no_punc'][i] = re.sub(newline_re2, ' ', (datadf['no_punc'][i]))
    datadf['no_punc'][i] = re.sub('\s+', ' ', datadf['no_punc'][i]).strip()

In [8]:
datadf = datadf.sort_values(by='date_journal')

In [9]:
for i in range(len(datadf['tags'])):
    if datadf['tags'][i]==[]:
        datadf['tags'][i]=['personal']

In [10]:
datadf['is_personal'] = 0
for x in range(len(datadf['tags'])):
    if ('personal' in str(datadf['tags'].iloc[x])):
        datadf['is_personal'].iloc[x] = 1

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
datadf

,text,date_modified,date_journal,address,lat,lon,mood,timezone,favourite,weather,photos,tags,no_punc,is_personal
id,,,,,,,,,,,,,,
1439137473084-3fc756cc3f8d6404,"The first thing I saw ,when I entered her room...",2017-08-31 01:44:57,2015-08-09 11:24:33,,1.79769e+308,1.79769e+308,0,,False,"{'id': -1, 'degree_c': 1.7976931348623157e+308...",[],"[personal, published]",The first thing I saw when I entered her room ...,1
1439198245820-3fda14add91e3f20,Philosophy\n\n\n\n\n\n\nIt surprises me..hw u ...,2017-08-22 22:08:05,2015-08-10 04:17:25,,1.79769e+308,1.79769e+308,0,,False,"{'id': -1, 'degree_c': 1.7976931348623157e+308...",[],[personal],Philosophy It surprises me hw u so effortlessl...,1
1439198461993-3fefd6db711e0342,Me \n\n\nWrite fantasies/inaginary dream world...,2017-10-08 12:24:49,2015-08-10 04:21:01,,1.79769e+308,1.79769e+308,0,,False,"{'id': -1, 'degree_c': 1.7976931348623157e+308...",[],[me],Me Write fantasies inaginary dream world stori...,0
1439575035515-3feb330512548cbd,"The dwarfs garden\n\nHe had a garden ,that lit...",2016-04-29 05:23:44,2015-08-14 12:57:15,,1.79769e+308,1.79769e+308,0,,False,"{'id': -1, 'degree_c': 1.7976931348623157e+308...",[],[published],The dwarfs garden He had a garden that little ...,0
1440393646546-3fdb2bc02b9f8020,Brigadier Suraj singh\n\nMy name is brigadier ...,2017-08-18 05:27:31,2015-08-24 00:20:46,,1.79769e+308,1.79769e+308,0,,False,"{'id': -1, 'degree_c': 1.7976931348623157e+308...",[],[script],Brigadier Suraj singh My name is brigadier sur...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589599203833-3fccc2487ff20d88,"<p dir=""auto"">That I really work well with int...",2020-05-15 22:20:45,2020-05-15 22:20:04,"412 Cedar Crest, Coppell, TX 75019, USA",32.971,-96.9803,1,America/Chicago,False,"{'id': 0, 'degree_c': 19.4, 'description': 'Ov...",[],[personal],That I really work well with introverts and pr...,1
1589599469199-3fc8b4cdb6d4b0d4,"<p dir=""auto""><span>Jodi Thor Dak Shoone Ko Na...",2020-05-15 22:49:11,2020-05-15 22:24:29,"412 Cedar Crest, Coppell, TX 75019, USA",32.971,-96.9803,1,America/Chicago,False,"{'id': 0, 'degree_c': 19.4, 'description': 'Ov...",[],[personal],Jodi Thor Dak Shoone Ko Na Ashi Tobe Ekla Chol...,1
1589603430128-3fdcc9774f183676,"<p dir=""auto"">It's really been a while since I...",2020-05-15 23:31:03,2020-05-15 23:30:30,"412 Cedar Crest, Coppell, TX 75019, USA",32.971,-96.9803,1,America/Chicago,False,"{'id': 0, 'degree_c': 19.4, 'description': 'Li...",[],[personal],It s really been a while since I ve told a gir...,1


In [12]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"hows": "how is",   
"I'd": "I would",
"I'll": "I will",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it shall / it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you're": "you are",
"you've": "you have"
}


def cleaner (test):
    test = re.sub('\\n', ' ', test)
    test = re.sub('<.*?>', ' ', test)
    test = re.sub('&nbsp;', ' ', test)
    test = re.sub('&[a-z]+;', ' ', test)
    test = re.sub('\.{2,}', ' ... ', test)
    test = re.sub('!{2,}', ' ! ', test)
    test = re.sub('\?{2,}', ' ? ', test)
    test = re.sub("’", "'", test)
    test = re.sub(" ' ", "'", test)
    test = re.sub("([^A-Za-z0-9'])", r" \1 ", test)
    test = re.sub('\s+', ' ', test)
    test = re.sub('\. \. \.', '...', test)

    test = re.sub(" i ", ' I ', test)
    test = re.sub(" u ", ' you ', test)
    test = re.sub("( im )|( Im )", " I'm ", test)
    test = re.sub(" ill ", " I'll ", test)
    test = re.sub("( ive )|( I ve )|( i ve )", " I've ", test)
    test = re.sub("( adi )|( Aditya )|( aditya )", " Adi ", test)

    uncontracted = []
    for x in test.split():
        if x.istitle():
            if x.lower() in contractions.keys():
                uncontracted.append(contractions[x.lower()].title())
            else:
                uncontracted.append(x)
        elif x.lower() in contractions.keys():
                uncontracted.append(contractions[x.lower()])
        elif x in contractions.keys():
                uncontracted.append(contractions[x])
        else:
            uncontracted.append(x)
        
        
    return(" ".join(uncontracted))


In [13]:
cleaner(datadf.text.loc['1586737092738-3fa9f1f5ffbc2dc0'])

'I wrote a haiku for mumma - You waited bland hours , To give me black walnut fudge " it is for energy " Inspired by Ada Limon\'s poem The Raincoat from the book The Carrying - The Raincoat Ada Lim n - 1976 - When the doctor suggested surgery and a brace for all my youngest years , my parents scrambled to take me to massage therapy , deep tissue work , osteopathy , and soon my crooked spine unspooled a bit , I could breathe again , and move more in a body unclouded by pain . My mom would tell me to sing songs to her the whole forty - five minute drive to Middle Two Rock Road and forty - five minutes back from physical therapy . She d say , even my voice sounded unfettered by my spine afterward . So I sang and sang , because I thought she liked it . I never asked her what she gave up to drive me , or how her day was before this chore . Today , at her age , I was driving myself home from yet another spine appointment , singing along to some maudlin but solid song on the radio , and I saw

In [14]:
datadf['cleaned_text'] = datadf['text'].apply(cleaner)

In [15]:
datadf.to_csv(r'All Data/Raw Journey Data/Until16May2020(all jsons to csv).csv')

In [16]:
tidy_format = datadf.no_punc.str.split(expand=True).stack().to_frame().reset_index(level=1)
tidy_format['num']=tidy_format['level_1']
tidy_format['word']=tidy_format[0]
tidy_format = tidy_format.drop(columns=[0,'level_1'])
tidy_format['lower'] =tidy_format['word'].str.lower()
afinn = pd.read_csv('All Data/AFINN-111.txt', sep='\t', header=None).rename(columns={0:'word',1:'afinn_score'})
afinn = afinn.set_index('word').to_dict()['afinn_score']
tidy_format['afinn_score'] = list(map(afinn.get, list(tidy_format.lower)))
tidy_format['afinn_score'] = tidy_format['afinn_score'].fillna(0)
tidy_format

,num,word,lower,afinn_score
id,,,,
1439137473084-3fc756cc3f8d6404,0,The,the,0.0
1439137473084-3fc756cc3f8d6404,1,first,first,0.0
1439137473084-3fc756cc3f8d6404,2,thing,thing,0.0
1439137473084-3fc756cc3f8d6404,3,I,i,0.0
1439137473084-3fc756cc3f8d6404,4,saw,saw,0.0
...,...,...,...,...
1589680083127-3fd337d30df0a5ce,115,story,story,0.0
1589680083127-3fd337d30df0a5ce,116,is,is,0.0
1589680083127-3fd337d30df0a5ce,117,above,above,0.0


In [17]:
tidy_format.to_csv('All Data/Raw Journey Data/TidyFormatUntil16May2020.csv')

In [18]:
word_counts = tidy_format['lower'].value_counts().to_frame().rename(columns={'lower':'count'})
word_counts.to_csv(r'All Data/Raw Journey Data/Word_Counts_Until16May2020.csv')
word_counts_dict = word_counts.to_dict()['count']
word_counts

,count
i,11480
the,8713
to,7859
and,7130
a,5748
...,...
distinguish,1
wom,1
hein,1
accordingly,1


In [19]:
timedata = pd.DataFrame(datadf[['date_journal']])
# timedata=timedata.set_index('id')
timedata['year'] = pd.DatetimeIndex(timedata['date_journal']).year
timedata['month'] = pd.DatetimeIndex(timedata['date_journal']).month
timedata['day'] = pd.DatetimeIndex(timedata['date_journal']).day
timedata['hour'] = pd.DatetimeIndex(timedata['date_journal']).hour
timedata['minute'] = pd.DatetimeIndex(timedata['date_journal']).minute
timedata['dayofweek'] = pd.DatetimeIndex(timedata['date_journal']).dayofweek
timedata = timedata.sort_values(by='date_journal')
timedata['day_num'] = (((pd.DataFrame(pd.DatetimeIndex(timedata.date_journal).date - 
                                      dt.datetime.strptime(timedata.date_journal[0], 
                                                           '%Y-%m-%d %H:%M:%S').date())[0])/np.timedelta64(1, 'D')).astype(int)).values
timedata['month_num'] = ((timedata['year'] - timedata['year'][0]) *12 ) + timedata['month'] - timedata['month'][0]
mapping_names = {1:'jan',2:'feb',3:'mar',4:'april',5:'may',6:'june',7:'july',8:'aug',9:'sept',10:'oct',11:'nov',12:'dec'}
timedata['month_name'] = timedata.date_journal.apply(lambda i:dt.datetime.strptime(i,'%Y-%m-%d %H:%M:%S').month).map(mapping_names)
timedata['year_month'] = timedata['year'].astype(str) + " " +timedata["month_name"]
timedata['minute_of_day'] = timedata['hour']*60 + timedata['minute']
timedata.to_csv('All Data/Raw Journey Data/TimeDataUntil16May2020.csv')

In [20]:
timedata

,date_journal,year,month,day,hour,minute,dayofweek,day_num,month_num,month_name,year_month,minute_of_day
id,,,,,,,,,,,,
1439137473084-3fc756cc3f8d6404,2015-08-09 11:24:33,2015,8,9,11,24,6,0,0,aug,2015 aug,684
1439198245820-3fda14add91e3f20,2015-08-10 04:17:25,2015,8,10,4,17,0,1,0,aug,2015 aug,257
1439198461993-3fefd6db711e0342,2015-08-10 04:21:01,2015,8,10,4,21,0,1,0,aug,2015 aug,261
1439575035515-3feb330512548cbd,2015-08-14 12:57:15,2015,8,14,12,57,4,5,0,aug,2015 aug,777
1440393646546-3fdb2bc02b9f8020,2015-08-24 00:20:46,2015,8,24,0,20,0,15,0,aug,2015 aug,20
...,...,...,...,...,...,...,...,...,...,...,...,...
1589599203833-3fccc2487ff20d88,2020-05-15 22:20:04,2020,5,15,22,20,4,1741,57,may,2020 may,1340
1589599469199-3fc8b4cdb6d4b0d4,2020-05-15 22:24:29,2020,5,15,22,24,4,1741,57,may,2020 may,1344
1589603430128-3fdcc9774f183676,2020-05-15 23:30:30,2020,5,15,23,30,4,1741,57,may,2020 may,1410


In [21]:
tidy_format

,num,word,lower,afinn_score
id,,,,
1439137473084-3fc756cc3f8d6404,0,The,the,0.0
1439137473084-3fc756cc3f8d6404,1,first,first,0.0
1439137473084-3fc756cc3f8d6404,2,thing,thing,0.0
1439137473084-3fc756cc3f8d6404,3,I,i,0.0
1439137473084-3fc756cc3f8d6404,4,saw,saw,0.0
...,...,...,...,...
1589680083127-3fd337d30df0a5ce,115,story,story,0.0
1589680083127-3fd337d30df0a5ce,116,is,is,0.0
1589680083127-3fd337d30df0a5ce,117,above,above,0.0


In [22]:
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [23]:
personal_stopwords = ["i've", "i'm","i'll", "like", "i'd", "he's", "she's", "he'd", "she'd"]

stopwords.extend(personal_stopwords)

In [24]:
stopwords = list(map(lambda x: re.sub('\'','',x), stopwords))

In [25]:
no_stop_tidy = tidy_format[(tidy_format.lower.apply(lambda x: x not in stopwords)).values]
# no_stop_tidy.to_csv('All Data/Raw Journey Data/NoStopTidy_Until16May2020.csv')

datadf = no_stop_tidy.groupby('id').agg({'word': lambda x : ' '.join(x)}).merge(datadf, on='id', how='right')
datadf = datadf.rename(columns = {'word':'no_stop'})

In [26]:
# going to try and clean textual data with nltk

In [27]:
from nltk.stem import WordNetLemmatizer as wnl
lemmatizer = wnl()

In [28]:
clean_text = datadf[['cleaned_text']]

In [29]:
clean_text['tokens'] = clean_text['cleaned_text'].apply(nltk.word_tokenize)

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
clean_text

,cleaned_text,tokens
id,,
1439137473084-3fc756cc3f8d6404,"The first thing I saw , when I entered her roo...","[The, first, thing, I, saw, ,, when, I, entere..."
1439198245820-3fda14add91e3f20,Philosophy It surprises me ... hw you so effor...,"[Philosophy, It, surprises, me, ..., hw, you, ..."
1439198461993-3fefd6db711e0342,Me Write fantasies / inaginary dream world sto...,"[Me, Write, fantasies, /, inaginary, dream, wo..."
1439575035515-3feb330512548cbd,"The dwarfs garden He had a garden , that littl...","[The, dwarfs, garden, He, had, a, garden, ,, t..."
1440393646546-3fdb2bc02b9f8020,Brigadier Suraj singh My name is brigadier sur...,"[Brigadier, Suraj, singh, My, name, is, brigad..."
...,...,...
1572234096800-3fe09f3bcb856ceb,,[]
1572394588516-3fe449570c8486e7,,[]
1573136070396-3feded4b7c9ee046,,[]


In [31]:
clean_text['pos'] = clean_text.tokens.apply(lambda x: nltk.pos_tag(x))

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [33]:
clean_text['lemm'] = clean_text.cleaned_text.apply(lemmatize_sentence)

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
clean_text

,cleaned_text,tokens,pos,lemm
id,,,,
1439137473084-3fc756cc3f8d6404,"The first thing I saw , when I entered her roo...","[The, first, thing, I, saw, ,, when, I, entere...","[(The, DT), (first, JJ), (thing, NN), (I, PRP)...","The first thing I saw , when I enter her room ..."
1439198245820-3fda14add91e3f20,Philosophy It surprises me ... hw you so effor...,"[Philosophy, It, surprises, me, ..., hw, you, ...","[(Philosophy, NN), (It, PRP), (surprises, VBZ)...",Philosophy It surprise me ... hw you so effort...
1439198461993-3fefd6db711e0342,Me Write fantasies / inaginary dream world sto...,"[Me, Write, fantasies, /, inaginary, dream, wo...","[(Me, NNP), (Write, NNP), (fantasies, NNS), (/...",Me Write fantasy / inaginary dream world stori...
1439575035515-3feb330512548cbd,"The dwarfs garden He had a garden , that littl...","[The, dwarfs, garden, He, had, a, garden, ,, t...","[(The, DT), (dwarfs, NN), (garden, NN), (He, P...","The dwarf garden He have a garden , that littl..."
1440393646546-3fdb2bc02b9f8020,Brigadier Suraj singh My name is brigadier sur...,"[Brigadier, Suraj, singh, My, name, is, brigad...","[(Brigadier, NNP), (Suraj, NNP), (singh, VB), ...",Brigadier Suraj singh My name be brigadier sur...
...,...,...,...,...
1572234096800-3fe09f3bcb856ceb,,[],[],
1572394588516-3fe449570c8486e7,,[],[],
1573136070396-3feded4b7c9ee046,,[],[],


In [35]:
tokens = clean_text.pos.explode().to_frame()

In [36]:
tokens['lemm'] = clean_text.lemm.apply(nltk.word_tokenize).explode()

In [37]:
tokens['word'] = clean_text.tokens.explode()
tokens['num'] = range(len(tokens))

In [38]:
tokens['lower'] = tokens['word'].str.lower()

In [39]:
temp = tokens[tokens.word.isna()]

In [40]:
tokens = tokens[tokens.pos.notna()]

In [41]:
x, tokens['pos_sep'] = zip(*list(tokens['pos']))

In [42]:
tokens = pd.concat([temp, tokens]).sort_values(by='num')

In [43]:
tokens

,pos,lemm,word,num,lower,pos_sep
id,,,,,,
1439137473084-3fc756cc3f8d6404,"(The, DT)",The,The,0,the,DT
1439137473084-3fc756cc3f8d6404,"(first, JJ)",first,first,1,first,JJ
1439137473084-3fc756cc3f8d6404,"(thing, NN)",thing,thing,2,thing,NN
1439137473084-3fc756cc3f8d6404,"(I, PRP)",I,I,3,i,PRP
1439137473084-3fc756cc3f8d6404,"(saw, VBD)",saw,saw,4,saw,VBD
...,...,...,...,...,...,...
1572234096800-3fe09f3bcb856ceb,NaN,NaN,NaN,328877,NaN,NaN
1572394588516-3fe449570c8486e7,NaN,NaN,NaN,328878,NaN,NaN
1573136070396-3feded4b7c9ee046,NaN,NaN,NaN,328879,NaN,NaN


In [44]:
tokens.to_csv('All Data/Raw Journey Data/TokensUntil16May2020.csv')

In [45]:
no_stop_lemm = tokens.dropna()[(tokens.lemm.str.lower().dropna().apply(lambda x: x not in stopwords)).values]

no_stop_lemm = no_stop_lemm.groupby('id').agg({'word': lambda x : ' '.join(x)})

In [48]:
core_text = no_stop_lemm.word.apply(lambda x: re.sub("[^a-zA-Z\s]", '', x).lower()).to_frame()
core_text.rename(columns={'word':'text'}, inplace=True)
core_text

,text
id,
1439137473084-3fc756cc3f8d6404,first thing saw entered room dark curtains ...
1439198245820-3fda14add91e3f20,philosophy surprises hw effortlessly make top...
1439198461993-3fefd6db711e0342,write fantasies inaginary dream world stories...
1439575035515-3feb330512548cbd,dwarfs garden garden little boy guarded rose...
1440393646546-3fdb2bc02b9f8020,brigadier suraj singh name brigadier suraj sin...
...,...
1589599203833-3fccc2487ff20d88,really work well introverts prefer close frien...
1589599469199-3fc8b4cdb6d4b0d4,jodi thor dak shoone ko na ashi tobe ekla chol...
1589603430128-3fdcc9774f183676,really since told girl love


In [49]:
core_text.to_csv('All Data/Raw Journey Data/Core_Text_Until16May2020.csv')